# Sensor-based Trip Safety Prediction

Predicting dangerous driving (positive label) from series of sensor readings

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import featuretools as ft
import featuretools.variable_types as vtypes
from featuretools.variable_types import Numeric
from featuretools.primitives import make_agg_primitive
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc
from scipy import interp

%matplotlib inline

## Dataset exploration

Explore labels

In [ ]:
labels_df = pd.read_csv('labels/part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv')
labels_df.head()

Check for duplicate values

In [ ]:
labels_df.groupby(['bookingID']).filter(lambda df:df.shape[0] > 1).sort_values(by=['bookingID'])

Duplicated detected. Ignoring them to avoid ambiguation

In [ ]:
labels_df = labels_df[~labels_df.duplicated(['bookingID'])]

In [ ]:
label_count = labels_df.label.value_counts()
print(label_count)
print('Negative ratio: {:.2f}'.format(label_count[0] * 1.0 / labels_df.shape[0]))
label_count.plot.bar()

> unbalanced dataset case

Explore data sample

In [ ]:
%%time

features_p1_df = pd.read_csv('features/part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
features_p1_df.head()

In [ ]:
features_p1_df.describe()

Visualize distributions of number of records per bookingID of **positive** (safe driving) vs **negative** (dangerous driving)

In [ ]:
plt.xlabel('Num of records')
plt.ylabel('Num of bookingID')
labels_df.loc[labels_df['label'] == 0].merge(features_p1_df, on='bookingID').bookingID.value_counts().hist(range=[0, 300], edgecolor='gray', color='blue', figsize=[10, 7])
labels_df.loc[labels_df['label'] == 1].merge(features_p1_df, on='bookingID').bookingID.value_counts().hist(range=[0, 300], edgecolor='gray', color='red', figsize=[10, 7])

> plot above shows there is no obvious correlation between number of records (sensor readings) and driving safety

Compare a pair of random negative and positive sample

In [ ]:
random_neg_bookingID = labels_df.loc[labels_df['label'] == 0].bookingID.sample(1).values[0]
random_neg_df = features_p1_df.loc[features_p1_df['bookingID'] == random_neg_bookingID].sort_values(by='second')

random_pos_bookingID = labels_df.loc[labels_df['label'] == 1].bookingID.sample(1).values[0]
random_pos_df = features_p1_df.loc[features_p1_df['bookingID'] == random_pos_bookingID].sort_values(by='second')

In [ ]:
random_neg_df.plot(title='Safe', kind='line',x='second',y=['Speed', 'Accuracy'], ylim=[0, 800], figsize=[10, 5])

random_pos_df.plot(title='Unsafe', kind='line',x='second',y=['Speed', 'Accuracy'], ylim=[0, 800], figsize=[10, 5])

In [ ]:
random_neg_df.plot(title='Safe', kind='line',x='second',y=['acceleration_x', 'acceleration_y', 'acceleration_z'], ylim=[-15, 15], figsize=[10, 5])
random_pos_df.plot(title='Unsafe', kind='line',x='second',y=['acceleration_x', 'acceleration_y', 'acceleration_z'], ylim=[-15, 15], figsize=[10, 5])

## Feature Engineering

Generate statistical features for each bookingID

In [ ]:
def mean_diff(column):
    return column.diff().mean(skipna=True)

def max_diff(column):
    return column.diff().max(skipna=True)

def min_diff(column):
    return column.diff().min(skipna=True)

def std_diff(column):
    return column.diff().std(skipna=True)

def mean_diff_abs(column):
    return column.diff().abs().mean(skipna=True)

def max_diff_abs(column):
    return column.diff().abs().max(skipna=True)

def min_diff_abs(column):
    return column.diff().abs().min(skipna=True)

def std_diff_abs(column):
    return column.diff().abs().std(skipna=True)


# data = pd.Series([1,2,7,3,5,10,3,1])
# print(avg_diff(data))
# print(std_diff(data))

In [ ]:
%%time

def get_feature_matrix(df):
    es = ft.EntitySet('safety_data')
    es.entity_from_dataframe(
        entity_id='records',
        index='id',        
        make_index=True,
        dataframe=df,        
        variable_types={
            'Accuracy': vtypes.Numeric,
            'Bearing': vtypes.Numeric,
            'acceleration_x': vtypes.Numeric,
            'acceleration_y': vtypes.Numeric,
            'acceleration_z': vtypes.Numeric,
            'gyro_x': vtypes.Numeric,
            'gyro_y': vtypes.Numeric,
            'gyro_z': vtypes.Numeric,
            'second': vtypes.Numeric,
            'Speed': vtypes.Numeric,
        }
    )
        
    es.normalize_entity(
        base_entity_id='records',
        new_entity_id='bookings',
        index='bookingID'    
    )
    
    print(es)
    
    return ft.dfs(
        entityset=es,
        target_entity='bookings',
        agg_primitives=[
            make_agg_primitive(function=mean_diff, input_types=[Numeric], return_type=Numeric),
            make_agg_primitive(function=max_diff, input_types=[Numeric], return_type=Numeric),
            make_agg_primitive(function=min_diff, input_types=[Numeric], return_type=Numeric),
            make_agg_primitive(function=std_diff, input_types=[Numeric], return_type=Numeric),
            make_agg_primitive(function=mean_diff_abs, input_types=[Numeric], return_type=Numeric),
            make_agg_primitive(function=max_diff_abs, input_types=[Numeric], return_type=Numeric),
            make_agg_primitive(function=min_diff_abs, input_types=[Numeric], return_type=Numeric),
            make_agg_primitive(function=std_diff_abs, input_types=[Numeric], return_type=Numeric),            
            'count', 
            'mean', 
            'max', 
            'min', 
            'std',
        ],
        n_jobs=1,
        verbose=True
    )

feature_matrix, feature_names = get_feature_matrix(features_p1_df.copy())
feature_matrix.head()

In [ ]:
feature_names

In [ ]:
feature_labels = feature_matrix.merge(labels_df, on='bookingID')['label']
print(feature_matrix.shape)
print(feature_labels.shape)

## Evaluation

Feature importances

In [ ]:
%%time

X = feature_matrix.values
y = feature_labels.values

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X, y)

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(len(feature_names)):
    top_feat = "{}. {}: {:.4g}".format(f + 1, feature_names[indices[f]].generate_name(), importances[indices[f]])
    print(top_feat)

ROC AUC scores

In [ ]:
%%time

# preprocess
# print('Scaling..')
# scaler = MinMaxScaler()
# X_scaled = scaler.fit_transform(X)
X_scaled = X

cv = StratifiedKFold(n_splits=5)
classifier = XGBClassifier(
    objective='binary:logistic', 
    eval_metric='auc',
    random_state=0
)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

i = 0
for train, test in cv.split(X_scaled, y):
    probas_ = classifier.fit(X_scaled[train], y[train]).predict_proba(X_scaled[test])
    fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
    tprs.append(interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b', label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()